In [198]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime
import pymongo

# Steam developer API key
from config import steam_api

# Steam List of All DLC

In [72]:
# Build the endpoint URL
base_url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/?key="
applist_url = base_url + steam_api

data = requests.get(applist_url).json()

dlc_list_df = pd.DataFrame(data['applist']['apps'])
dlc_list_df.sort_values(by=['appid']).to_csv('dlc_list_steam.csv', encoding='utf-8-sig', index = False)

In [76]:
# After saving the completed list of downloadable content from Steam, read the file and display the DF.
dlc_list = pd.read_csv('dlc_list_steam.csv')
dlc_list


,appid,name
0,5,Dedicated Server
1,7,Steam Client
2,8,winui2
3,10,Counter-Strike
4,20,Team Fortress Classic
...,...,...
107563,1511250,Vector Race
107564,1511610,Gang wars
107565,1511620,Dark spot
107566,1512070,VR Hiroshima 1945


# SteamSpy API of All Steam 'Games' with Metadata

###  Steam allows 200 api requests every 5 minutes.  We are only interested in 'games', not other content such as soundtracks or packs, so we used SteamSpy's api to only the list of just Steam's games.

In [21]:
# Create an empty list to store the 45 pages of dictionaries.
frames_list = []

for page in range(0, 45):
    # Url for api requests
    steamspy_url = f'https://steamspy.com/api.php?request=all&page={page}'
    
    data = requests.get(steamspy_url).json()

    # Append each dictionary to the list
    frames_list.append(pd.DataFrame.from_dict(data, orient='index'))

# Concatenate the list of dictionaries to create a DataFrame
df = pd.concat(frames_list)   
steamspy_final = df.sort_values('appid').reset_index().drop(columns='index')
steamspy_final.head(4)

# del(frames_list)


,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,320,Half-Life 2: Deathmatch,Valve,Valve,,8133,927,0,"500,000 .. 1,000,000",493,0,42,0,99,499,80,339
1,852,ValveTestApp852,,,,0,0,0,"0 .. 20,000",0,0,0,0,0,0,0,0
2,1200,Red Orchestra: Ostfront 41-45,Tripwire Interactive,Tripwire Interactive,,1807,255,0,"200,000 .. 500,000",539,21,556,21,249,499,50,51
3,1309,SiN Episodes: Emergence,Ritual Entertainment,Ritual Entertainment,,322,45,0,"200,000 .. 500,000",0,0,0,0,249,999,75,0


In [5]:
steamspy_final.to_csv('steamspy_final.csv', encoding='utf-8-sig', index = False)


# Steam API Call for Metadata of All 'Games'

In [178]:
# Read the SteamSpy csv list of Steam games to make the api requests through Steam
steamspy_final = pd.read_csv('steamspy_final.csv')

In [25]:
# Url for api requests and the list of appids/games to loop through
url = f"http://store.steampowered.com/api/appdetails/?appids="
appids = steamspy_final['appid'].tolist()

# Empty lists of desired keys
dlc_type = []
name = []
steam_appid = []
short_description = []
metascore = []
categories = []
genres = []
recommendations = []
release_date = []

# For Loop to capture the keys and values of the dictionaries from the api requests
for game_id in appids:
    try:
        query_url = url + str(game_id)
        data_steam = requests.get(query_url).json()[str(game_id)]['data']
        
        dlc_type.append(data_steam['type'] if data_steam.get('type') else '')
        name.append(data_steam['name'] if data_steam.get('name') else '')
        steam_appid.append(data_steam['steam_appid'] if data_steam.get('steam_appid') else '')
        short_description.append(data_steam['short_description'] if data_steam.get('short_description') else '')
        metascore.append(data_steam['metacritic']['score'] if data_steam.get('metacritic') else '')
        categories.append(", ".join([x['description'] for x in data_steam.get('categories', "")]))
        genres.append(", ".join([x['description'] for x in data_steam.get('genres', "")]))
        recommendations.append(data_steam['recommendations']['total'] if data_steam.get('recommendations') else '')
        release_date.append(data_steam['release_date']['date'] if data_steam.get('release_date') else '')
        
        # Time set to cap the requests to 200 every 5 minutes
        time.sleep(1.5)
        
#Print any appids that fail and their exceptions.   
    except Exception as e:
        print(f'Skipping app: {game_id}')
        print(e)


Skipping app: 852
'data'
Skipping app: 8740
'data'
Skipping app: 8955
'data'
Skipping app: 13120
'data'
Skipping app: 13260
'data'
Skipping app: 17760
'data'
Skipping app: 18310
'data'
Skipping app: 22490
'data'
Skipping app: 23130
'data'
Skipping app: 23140
'data'
Skipping app: 25730
'data'
Skipping app: 29017
'data'
Skipping app: 35470
'data'
Skipping app: 41310
'data'
Skipping app: 46470
'data'
Skipping app: 50400
'data'
Skipping app: 55130
'data'
Skipping app: 55130
'data'
Skipping app: 55190
'data'
Skipping app: 55210
'data'
Skipping app: 63230
'data'
Skipping app: 71300
'data'
Skipping app: 72520
'data'
Skipping app: 72530
'data'
Skipping app: 90800
'data'
Skipping app: 98810
'data'
Skipping app: 99110
'data'
Skipping app: 99930
'data'
Skipping app: 102210
'data'
Skipping app: 201930
'data'
Skipping app: 209340
'data'
Skipping app: 209340
'data'
Skipping app: 212090
'data'
Skipping app: 214400
'data'
Skipping app: 215490
'data'
Skipping app: 215590
'data'
Skipping app: 221680
Exp

In [184]:
# Create a dictionary of the keys and values pulled from the api request and create a Dataframe. 
game_dict = {
    'steam_appid': steam_appid,
    'type': dlc_type,
    'name': name,
    'short_description': short_description,
    'metascore': metascore,
    'categories': categories,
    'genres': genres,
    'recommendations': recommendations,
    'release_date': release_date
}

steam_game_df = pd.DataFrame(game_dict)
steam_game_df.head(4) 



,steam_appid,type,name,short_description,metascore,categories,genres,recommendations,release_date
0,320,game,Half-Life 2: Deathmatch,Fast multiplayer action set in the Half-Life 2...,,"Multi-player, Valve Anti-Cheat enabled, Includ...",Action,5733,"Nov 1, 2004"
1,1200,game,Red Orchestra: Ostfront 41-45,Fight in the theatre of war that changed the w...,81,"Multi-player, Steam Achievements, Valve Anti-C...",Action,1135,"Mar 14, 2006"
2,1300,game,SiN Episodes: Emergence,"You are John Blade, commander of HardCorps, an...",75,"Single-player, Stats",Action,417,"May 10, 2006"
3,1313,game,SiN: Gold,SiN: Gold has returned! Free update for origin...,,"Single-player, Multi-player, PvP, Online PvP, ...",Action,130,"Mar 18, 2020"


In [82]:
steam_game_df.to_csv('steam_game_api_data.csv', encoding='utf-8-sig', index = False)

# Merge SteamSpy dataframe with Steam's API dataframe

In [185]:
# reading the csv files from Steam and SteamSpy to clean and merge into a final dataframe.
steam_premerge = pd.read_csv('steam_game_api_data.csv').rename(columns={'steam_appid': 'appid'})

In [186]:
steam_clean = steam_premerge.drop_duplicates(subset=['appid'])

In [187]:
steamspy_clean = steamspy_final.drop_duplicates(subset=['appid'])


In [188]:
steam_games_merged = pd.merge(steam_clean, steamspy_clean, on="appid", how="left")
steam_games_merged

,appid,type,name_x,short_description,metascore,categories,genres,recommendations,release_date,name_y,...,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,10,game,Counter-Strike,Play the world's number 1 online action game. ...,88.0,"Multi-player, PvP, Online PvP, Shared/Split Sc...",Action,100006.0,1-Nov-00,Counter-Strike,...,0.0,"10,000,000 .. 20,000,000",7429.0,143.0,178.0,65.0,199.0,999.0,80.0,19836.0
1,20,game,Team Fortress Classic,One of the most popular online action games of...,NaN,"Multi-player, PvP, Online PvP, Shared/Split Sc...",Action,3754.0,1-Apr-99,Team Fortress Classic,...,0.0,"1,000,000 .. 2,000,000",162.0,1.0,24.0,1.0,99.0,499.0,80.0,122.0
2,30,game,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...,79.0,"Multi-player, Valve Anti-Cheat enabled",Action,2779.0,1-May-03,Day of Defeat,...,0.0,"5,000,000 .. 10,000,000",494.0,2.0,28.0,2.0,99.0,499.0,80.0,162.0
3,40,game,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,NaN,"Multi-player, PvP, Online PvP, Shared/Split Sc...",Action,1276.0,1-Jun-01,Deathmatch Classic,...,0.0,"5,000,000 .. 10,000,000",215.0,1.0,10.0,1.0,99.0,499.0,80.0,10.0
4,50,game,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...,NaN,"Single-player, Multi-player, Valve Anti-Cheat ...",Action,8162.0,1-Nov-99,Half-Life: Opposing Force,...,0.0,"2,000,000 .. 5,000,000",558.0,0.0,172.0,0.0,99.0,499.0,80.0,202.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33755,1499780,game,Beat The Moles,Beat The Moles is an arcade game where you hav...,NaN,Single-player,"Casual, Indie, Simulation",NaN,26-Dec-20,Beat The Moles,...,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,59.0,99.0,40.0,0.0
33756,1500260,game,Desert Mystery,"A hardcore tactical shooter, where your main m...",NaN,Single-player,"Action, Adventure, Casual, Indie, Strategy",NaN,28-Dec-20,Desert Mystery,...,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,99.0,99.0,0.0,1.0
33757,1500490,game,Teslapunk,An arcade shoot-em-up with a retro science-fic...,NaN,"Single-player, Full controller support, Steam ...",Action,NaN,30-Dec-20,Teslapunk,...,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,999.0,999.0,0.0,0.0
33758,1502790,game,Gun Witch,Help Beretta become the best witch ever in thi...,NaN,Single-player,"Action, Adventure, Casual, Free to Play, Indie...",NaN,31-Dec-20,Gun Witch,...,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0


In [189]:
steam_games_merged.dtypes

appid                  int64
type                  object
name_x                object
short_description     object
metascore            float64
categories            object
genres                object
recommendations      float64
release_date          object
name_y                object
developer             object
publisher             object
score_rank           float64
positive             float64
negative             float64
userscore            float64
owners                object
average_forever      float64
average_2weeks       float64
median_forever       float64
median_2weeks        float64
price                float64
initialprice         float64
discount             float64
ccu                  float64
dtype: object

In [190]:
steam_games_merged.to_csv('steam_games_merged_for_cleaning.csv', encoding='utf-8-sig', index = False)

## Transform

In [194]:
merged = pd.read_csv('steam_games_merged_for_cleaning.csv', encoding='utf-8-sig')

In [195]:
df1 = merged.drop(columns=['name_y', 'score_rank', 'userscore'])
steam_final_cleaned = df1.fillna(0)
steam_final_cleaned

,appid,type,name_x,short_description,metascore,categories,genres,recommendations,release_date,developer,...,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,10,game,Counter-Strike,Play the world's number 1 online action game. ...,88.0,"Multi-player, PvP, Online PvP, Shared/Split Sc...",Action,100006.0,1-Nov-00,Valve,...,4360.0,"10,000,000 .. 20,000,000",7429.0,143.0,178.0,65.0,1.99,9.99,80.0,19836.0
1,20,game,Team Fortress Classic,One of the most popular online action games of...,0.0,"Multi-player, PvP, Online PvP, Shared/Split Sc...",Action,3754.0,1-Apr-99,Valve,...,800.0,"1,000,000 .. 2,000,000",162.0,1.0,24.0,1.0,0.99,4.99,80.0,122.0
2,30,game,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...,79.0,"Multi-player, Valve Anti-Cheat enabled",Action,2779.0,1-May-03,Valve,...,493.0,"5,000,000 .. 10,000,000",494.0,2.0,28.0,2.0,0.99,4.99,80.0,162.0
3,40,game,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,0.0,"Multi-player, PvP, Online PvP, Shared/Split Sc...",Action,1276.0,1-Jun-01,Valve,...,353.0,"5,000,000 .. 10,000,000",215.0,1.0,10.0,1.0,0.99,4.99,80.0,10.0
4,50,game,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...,0.0,"Single-player, Multi-player, Valve Anti-Cheat ...",Action,8162.0,1-Nov-99,Gearbox Software,...,520.0,"2,000,000 .. 5,000,000",558.0,0.0,172.0,0.0,0.99,4.99,80.0,202.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33755,1499780,game,Beat The Moles,Beat The Moles is an arcade game where you hav...,0.0,Single-player,"Casual, Indie, Simulation",0.0,26-Dec-20,INFINITY BRIDGE,...,1.0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.59,0.99,40.0,0.0
33756,1500260,game,Desert Mystery,"A hardcore tactical shooter, where your main m...",0.0,Single-player,"Action, Adventure, Casual, Indie, Strategy",0.0,28-Dec-20,"Cosmin, Josh",...,1.0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.99,0.99,0.0,1.0
33757,1500490,game,Teslapunk,An arcade shoot-em-up with a retro science-fic...,0.0,"Single-player, Full controller support, Steam ...",Action,0.0,30-Dec-20,"klutzGames, Fleischfilm",...,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,9.99,9.99,0.0,0.0
33758,1502790,game,Gun Witch,Help Beretta become the best witch ever in thi...,0.0,Single-player,"Action, Adventure, Casual, Free to Play, Indie...",0.0,31-Dec-20,AGM Studios,...,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.00,0.00,0.0,12.0


In [196]:
steam_final_cleaned.to_csv('steam_final_cleaned.csv', encoding='utf-8-sig', index = False)

## Load

In [199]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [202]:
db.appid.drop()

In [201]:
# Define database and collection
db = client.steam_project_db
collection = db.appid

In [203]:
# Dictionary to be inserted as a MongoDB document
steam_dict = steam_final_cleaned.to_dict("records")

# Insert collection
collection.insert_many(steam_dict)

In [204]:
games = db.appid.find()

for game in games:
    print(game)

{'_id': ObjectId('5ff2e1d68912bfea9edd46b7'), 'appid': 10, 'type': 'game', 'name_x': 'Counter-Strike', 'short_description': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.", 'metascore': 88.0, 'categories': 'Multi-player, PvP, Online PvP, Shared/Split Screen PvP, Valve Anti-Cheat enabled', 'genres': 'Action', 'recommendations': 100006.0, 'release_date': '1-Nov-00', 'developer': 'Valve', 'publisher': 'Valve', 'positive': 169280.0, 'negative': 4360.0, 'owners': '10,000,000 .. 20,000,000', 'average_forever': 7429.0, 'average_2weeks': 143.0, 'median_forever': 178.0, 'median_2weeks': 65.0, 'price': 1.99, 'initialprice': 9.99, 'discount': 80.0, 'ccu': 19836.0}
{'_id': ObjectId('5ff2e1d68912bfea9edd46b8'), 'appid': 20, 'type': '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'_id': ObjectId('5ff2e1d68912bfea9edd8f03'), 'appid': 788100, 'type': 'game', 'name_x': 'Neon Abyss', 'short_description': 'Neon Abyss is a frantic, roguelite action-platformer where you run ‘n’ gun your way into the Abyss. Featuring unlimited item synergies and a unique dungeon evolution system, each run diversifies the experience and every choice alters the ruleset.', 'metascore': 0.0, 'categories': 'Single-player, Steam Achievements, Steam Trading Cards, Partial Controller Support, Steam Cloud, Remote Play on Phone, Remote Play on Tablet', 'genres': 'Action, Adventure, Indie, RPG', 'recommendations': 6479.0, 'release_date': '14-Jul-20', 'developer': 'Veewo Games', 'publisher': 'Team17 Digital Ltd, Yooreka Studio (China)', 'positive': 6372.0, 'negative': 1155.0, 'owners': '500,000 .. 1,000,000', 'average_forever': 1299.0, 'average_2weeks': 603.0, 'median_forever': 971.0, 'median_2weeks': 627.0, 'price': 14.99, 'initialprice': 19.99, 'discount': 25.0, 'ccu': 1333.0}
{'_id': ObjectId(

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [205]:
db.appid.count()

C:\Users\cch31\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


33760